# QM8 Dataset

In [1]:
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import numpy as np
import tensorflow as tf
import deepchem as dc
from deepchem.models.tensorgraph.models.graph_models import GraphConvTensorGraph
deepchem.molnet.load_function.qm8_datasets.load_qm_from_mat()

ModuleNotFoundError: No module named 'tensorflow'

## Multitask Network

## Kernel Ridge Regression

## Graph Convolution

## Multitask Network _(Coulomb Matrix Featurizer)_

## Kernel Ridge Regression _(Coulomb Matrix Featurizer)_

## Deep Tensor Neural Network

## Message Passing Neural Network